In [ ]:
!pip install shap transformers datasets

In [ ]:
import numpy as np
import pandas as pd
import shap
import pickle
import transformers
from transformers import ZeroShotClassificationPipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from typing import Union, List
from tqdm import tqdm
import datasets
import torch
import numpy as np
import scipy as sp
import os

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dfFinalArm.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roupenminassian/TwHIN-BERT-Misinformation-Classifier", use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained("roupenminassian/TwHIN-BERT-Misinformation-Classifier").cuda()

In [ ]:
def process_list(dataList,valueList):
  result = []
  word = []

  totalSum = []
  sum = 0

  count = 0
  for index, i in enumerate(dataList):
    if ' ' not in i:
      word.append(i)
      sum += valueList[index]
      count = 0
      if i == dataList[-1]:
        result.append(''.join(word))
        totalSum.append(str(sum))
        word = []
        sum = 0

    elif ' ' in i and count==0:
      word.append(i)
      sum += valueList[index]
      result.append(''.join(word))
      totalSum.append(str(sum))
      word = []
      sum = 0
      count = 1

    else:
      if i != dataList[1]:
        result.append(i)
        totalSum.append(str(valueList[index]))

  return result, totalSum

In [ ]:
def appendShap(df, model, tokenizer):
  shap_dictionary = []

  save_interval = 1
  pickle_file = 'SHAP.pickle'
  last_processed_index_file = 'last_processed_SHAP.pickle'

  # Load the last processed index from the file
  if os.path.exists(last_processed_index_file):
      with open(last_processed_index_file, 'rb') as f:
          start_index = pickle.load(f) + 1
  else:
      start_index = 0

  # Load the existing responses from the pickle file
  if os.path.exists(pickle_file):
      with open(pickle_file, 'rb') as f:
          try:
              label_class = pickle.load(f)
          except EOFError:
              pass


  classifier = pipeline(task="text-classification", model=model, tokenizer=tokenizer, device=0, top_k=None)
  explainer = shap.Explainer(classifier)

  for index, text in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=df.shape[0], desc="Processing rows"):
    shap_values = explainer([text["full_text"]])
    SHAPkeys, SHAPvalues = process_list(shap_values[0,:,'factual'].data, shap_values[0,:,'factual'].values)
    my_dictionary = {keys: values for keys, values in zip(SHAPkeys, SHAPvalues)}
    shap_dictionary.append(my_dictionary)

    # Save the responses every 100 entries
    if (index + 1) % save_interval == 0:
        with open(pickle_file, 'wb') as f:  # Open the file in write-binary mode
            pickle.dump(shap_dictionary, f)

        # Save the last processed index to a file
        with open(last_processed_index_file, 'wb') as f:
            pickle.dump(index, f)

  # Save the remaining responses and the last processed index
  with open(pickle_file, 'wb') as f:
      pickle.dump(label_class, f)

  with open(last_processed_index_file, 'wb') as f:
      pickle.dump(index, f)

  return shap_dictionary

In [ ]:
appendShap(df,model,tokenizer)